In [1]:
# Cell 1: Install required packages
!pip install torch torchvision torchaudio --quiet
!pip install timm facenet-pytorch albumentations opencv-python mediapipe kagglehub scikit-learn --quiet
!pip install --upgrade pip --quiet

## Imports


In [ ]:
# Cell 2: Import libraries
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from PIL import Image
from tqdm import tqdm
import kagglehub
from sklearn.utils.class_weight import compute_class_weight 

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


## Dataset Import


In [3]:
# Cell 3: Download RAF-DB via kagglehub
dataset_path = kagglehub.dataset_download("shuvoalok/raf-db-dataset")
print("Path to dataset files:", dataset_path)

data_root = os.path.join(dataset_path, 'DATASET')

Path to dataset files: /home/9826/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2


##  Define Label Mapping and Constants


In [ ]:
# Cell 4: Define Label Mapping and Constants


EMOTION_LABELS = {
    0: 'Surprise', 
    1: 'Fear',     
    2: 'Happy',     
    3: 'Sad',      
    4: 'Angry',    
    5: 'Neutral'   
}

BATCH_SIZE = 32
NUM_WORKERS = 2 
NUM_CLASSES = 6 

## Define Transforms (Standard ResNet Normalization)

In [5]:
# Cell 5: Define Transforms (Standard ResNet Normalization)
mean_vals = [0.485, 0.456, 0.406]
std_vals = [0.229, 0.224, 0.225]

train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Blur(blur_limit=3, p=0.1),
    A.Normalize(mean=mean_vals, std=std_vals),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=mean_vals, std=std_vals),
    ToTensorV2()
])

/home/9826/.local/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [ ]:
class RAFDataset(Dataset):
    def __init__(self, root_dir, phase='train', transform=None):
        self.img_dir = os.path.join(root_dir, phase)
        self.transform = transform
        self.img_paths = []
        self.labels = []

        # MAPPING (Skipping Disgust)
        self.folder_to_label = {
            '1': 0, # Surprise
            '2': 1, # Fear
            '4': 2, # Happy
            '5': 3, # Sad
            '6': 4, # Angry
            '7': 5  # Neutral
        }

        # OVERSAMPLING CONFIG
        # If the class is "Fear" (Folder 2) or "Angry" (Folder 6),
        # we repeat the images X times to force the model to learn them.
        self.oversample_factors = {
            '2': 5,  # Fear: Repeat 5 times (Crucial!)
            '6': 3,  # Angry: Repeat 3 times
            '5': 2   # Sad: Repeat 2 times
        }

        # Load Images
        available_folders = os.listdir(self.img_dir)
        for folder_name in available_folders:
            if folder_name in self.folder_to_label:
                label_idx = self.folder_to_label[folder_name]
                folder_path = os.path.join(self.img_dir, folder_name)

                # Determine how many times to repeat this folder
                # Default is 1 (no repeat)
                repeat_count = self.oversample_factors.get(folder_name, 1)
                if phase == 'test':
                    repeat_count = 1  # Never oversample test data!

                for img_file in os.listdir(folder_path):
                    if img_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                        full_path = os.path.join(folder_path, img_file)

                        # THE HACK: Add the same file multiple times
                        self.img_paths.extend([full_path] * repeat_count)
                        self.labels.extend([label_idx] * repeat_count)

        print(f"Loaded {len(self.img_paths)} images for {phase} (With Oversampling)")

## Model Initialization

In [7]:
# Cell 7: Model Initialization
print("Initializing ResNet50 for 6 Classes...")
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Modify the final layer for 6 classes (Not 7)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, NUM_CLASSES)

model = model.to(device)

Initializing ResNet50 for 6 Classes...


## Phase 1: Training Head

In [ ]:
# Cell 8: Phase 1 Training (Weighted Loss)

# 1. CALCULATE CLASS WEIGHTS TO FIX IMBALANCE
print("Calculating Class Weights to fix bias...")
# Extract all labels from the dataset
y_train = train_dataset.labels
classes = np.unique(y_train)

# Calculate weights: Rare classes (Fear, Angry) get HIGHER weights
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
weights_tensor = torch.tensor(weights, dtype=torch.float).to(device)

print(f"Class Weights applied: {weights}")
# You should see high numbers for Fear/Angry and low for Happy/Neutral

# 2. DEFINE LOSS WITH WEIGHTS
criterion = nn.CrossEntropyLoss(weight=weights_tensor)
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

# Freeze body, train head
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

num_epochs =  5
print("Starting Phase 1: Training Head (With Weights)...")

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels) # Weighted Loss used here
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        loop.set_postfix(loss=running_loss/len(train_loader), acc=100.*correct/total)

Starting Phase 1: Training Head...


Epoch 5/5: 100%|██████████| 384/384 [00:44<00:00,  8.56it/s, acc=52.8, loss=1.27] 


## Phase 2 Training (Fine-Tuning)

In [ ]:
# Cell 9: Phase 2 Training (Fine-Tuning)
print("Starting Phase 2: Fine-Tuning...")

# Unfreeze the last two blocks (layer3 and layer4)
for name, param in model.named_parameters():
    if 'layer3' in name or 'layer4' in name or 'fc' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Use a much lower learning rate for fine-tuning
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

num_epochs = 35 
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        loop.set_postfix(loss=running_loss/len(train_loader), acc=100.*correct/total)

# Save the model
torch.save(model.state_dict(), 'rafdb_resnet50_6classes_weighted.pth')
print("Model Saved!")

Starting Phase 2: Fine-Tuning...


Epoch 35/35: 100%|██████████| 384/384 [00:39<00:00,  9.68it/s, acc=98, loss=0.0568]   


Model Saved!
